# Nextstrain Zika tutorial
Instructions from https://nextstrain.org/docs/getting-started/zika-tutorial/

## Copy files from Nextstrain Github

In [1]:
# !git clone https://github.com/nextstrain/zika-tutorial.git

Cloning into 'zika-tutorial'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 27 (delta 0), reused 3 (delta 0), pack-reused 22
Unpacking objects: 100% (27/27), done.
Checking connectivity... done.


# Prepare the Sequences
## Filter the Sequences
Filter the parsed sequences and metadata to exclude strains from subsequent analysis and subsample the remaining strains to a fixed number of samples per group.

In [1]:
!mkdir -p zika-tutorial/results/

!augur filter \
  --sequences zika-tutorial/data/sequences.fasta \
  --metadata zika-tutorial/data/metadata.tsv \
  --exclude zika-tutorial/config/dropped_strains.txt \
  --output zika-tutorial/results/filtered.fasta \
  --group-by country year month \
  --sequences-per-group 20 \
  --min-date 2012


# For documentation of all `filter` commands
# !augur filter -h

### Documentation for each command
#### `!augur filter`


`--sequences zika-tutorial/data/sequences.fasta`: Sequences in fasta or VCF format




`--metadata zika-tutorial/data/metadata.tsv`: Metadata associated with sequences


`--exclude zika-tutorial/config/dropped_strains.txt`: File with list of strains that are to be excluded


`--output zika-tutorial/results/filtered.fasta`: Output file

`--group-by country year month`: Categories with respect to subsample; two virtual fields, "month" and "year", are supported if they don't already exist as real fields but a "date" field does exist.

**I don't quite understand what we're grouping for. Is this to tell it to group virus populations by their country > year > month? If so, why is this in 'filter' rather than the tree building pipeline?**


`--sequences-per-group 20`: Subsample to no more than this number of sequences per category

**I don't understand why the number of sequences per group matters**

`--min-date 2012`: Minimal cutoff for numerical date

## Align the Sequences
Create a multiple alignment of the sequences using a custom reference. After this alignment, columns with gaps in the reference are removed. Additionally, the --fill-gaps flag fills gaps in non-reference sequences with “N” characters. These modifications force all sequences into the same coordinate space as the reference sequence.

In [1]:
!augur align \
  --sequences zika-tutorial/results/filtered.fasta \
  --reference-sequence zika-tutorial/config/zika_outgroup.gb \
  --output zika-tutorial/results/aligned.fasta \
  --fill-gaps


using mafft to align via:
	mafft --reorder --anysymbol --thread 1 zika-tutorial/results/filtered.fasta.ref.fasta 1> zika-tutorial/results/aligned.fasta 2> zika-tutorial/results/aligned.fasta.log 

	Katoh et al, Nucleic Acid Research, vol 30, issue 14
	https://doi.org/10.1093%2Fnar%2Fgkf436

Trimmed gaps in KX369547.1 from the alignment


### Documentation for each command
#### `!augur align`

`--sequences zika-tutorial/results/filtered.fasta`

`--reference-sequence zika-tutorial/config/zika_outgroup.gb`: Strip insertions relative to reference sequence; use if the reference is NOT already in the input sequences

**"Strip insertions" doesn't make sense to me. Is this just to remove insertions? And we're telling it which sequence is the reference sequence?**

`--output zika-tutorial/results/aligned.fasta`

`--fill-gaps`: if gaps represent missing data rather than true indels, replace by N after aligning

# Construct the Phylogeny
Infer a phylogenetic tree from the multiple sequence alignment.

In [3]:
!augur tree \
  --alignment zika-tutorial/results/aligned.fasta \
  --output zika-tutorial/results/tree_raw.nwk

Building a tree via:
	iqtree -ninit 2 -n 2 -me 0.05 -nt 1 -s zika-tutorial/results/aligned-delim.fasta -m GTR > zika-tutorial/results/aligned-delim.iqtree.log
	Nguyen et al: IQ-TREE: A fast and effective stochastic algorithm for estimating maximum likelihood phylogenies.
	Mol. Biol. Evol., 32:268-274. https://doi.org/10.1093/molbev/msu300

Building original tree took 0.3931910991668701 seconds


### Documentation for each command
#### `!augur tree`

`!augur tree` can use [FastTree](http://www.microbesonline.org/fasttree/), [RAxML](https://sco.h-its.org/exelixis/web/software/raxml/index.html), or [IQ-TREE](http://www.iqtree.org/) (default)

**I haven't learned anything about how trees are constructed yet**

`--alignment zika-tutorial/results/aligned.fasta`: Alignment in fasta or VCF format

`--output zika-tutorial/results/tree_raw.nwk`

The resulting tree is stored in Newick format. Branch lengths in this tree measure nucleotide divergence.

## Get a Time-Resolved Tree
Augur can also adjust branch lengths in this tree to position tips by their sample date and infer the most likely time of their ancestors, using [TreeTime](https://github.com/neherlab/treetime). Run the `refine` command to apply TreeTime to the original phylogenetic tree and produce a “time tree”.

In [4]:
!augur refine \
  --tree zika-tutorial/results/tree_raw.nwk \
  --alignment zika-tutorial/results/aligned.fasta \
  --metadata zika-tutorial/data/metadata.tsv \
  --output-tree zika-tutorial/results/tree.nwk \
  --output-node-data zika-tutorial/results/branch_lengths.json \
  --timetree \
  --coalescent opt \
  --date-confidence \
  --date-inference marginal \
  --clock-filter-iqd 4

pruning leaf  KX369547.1

1.71	###TreeTime.run: INITIAL ROUND

6.45	###TreeTime.run: ITERATION 1 out of 2 iterations

14.28	###TreeTime.run: ITERATION 2 out of 2 iterations

44.42	###TreeTime.run: FINAL ROUND - confidence estimation via marginal reconstruction

Inferred a time resolved phylogeny using TreeTime:
	Sagulenko et al. TreeTime: Maximum-likelihood phylodynamic analysis
	Virus Evolution, vol 4, https://academic.oup.com/ve/article/4/1/vex042/4794731

updated tree written to zika-tutorial/results/tree.nwk
node attributes written to zika-tutorial/results/branch_lengths.json


### Documentation for each command
#### `!augur refine`
  `--tree zika-tutorial/results/tree_raw.nwk`
  
  `--alignment zika-tutorial/results/aligned.fasta`
  
  `--metadata zika-tutorial/data/metadata.tsv`
  
  `--output-tree zika-tutorial/results/tree.nwk`
  
  `--output-node-data zika-tutorial/results/branch_lengths.json`: File name to write branch lengths as node data
  
  `--timetree`: Produce timetree using treetime
  
  `--coalescent opt`: coalescent time scale in units of inverse clock rate (float), optimize as scalar ('opt'), or skyline ('skyline')
  
  **"coalescent" time scale?**
  
  `--date-confidence`: Calculate confidence intervals for node dates (default: False)
  
  `--date-inference marginal`: {joint,marginal} assign internal nodes to their marginally most likely dates, not jointly most likely (default: joint)
  
  **"Marginally" vs "jointly"?**
  
  `--clock-filter-iqd 4`: clock-filter: remove tips that deviate more than n_iqd interquartile ranges from the root-to-tip vs time regression (default: None)
  
  **If they're way too short or way too long?**
  
  
### Output file branch_lengths.json
For each node:
```
"ZKC2/2016": {
      "raw_date": "2016-02-16",
      "numdate": 2016.1286789869953,
      "num_date_confidence": [
        2016.1286789869953,
        2016.1286789869953
      ],
      "mutation_length": 1.3607603996744637e-22,
      "date": "2016-02-16",
      "clock_length": 6.091141628134303e-06,
      "branch_length": 6.091141628134303e-06
```

What does **mutation_length** mean?
      

In [23]:
!cat zika-tutorial/results/branch_lengths.json | jq ''

{
  "nodes": {
    "ZKC2/2016": {
      "raw_date": "2016-02-16",
      "numdate": 2016.1286789869953,
      "num_date_confidence": [
        2016.1286789869953,
        2016.1286789869953
      ],
      "mutation_length": 1.3607603996744637e-22,
      "date": "2016-02-16",
      "clock_length": 6.091141628134303e-06,
      "branch_length": 6.091141628134303e-06
    },
    "VEN/UF_1/2016": {
      "raw_date": "2016-03-25",
      "numdate": 2016.2327173169062,
      "num_date_confidence": [
        2016.2327173169062,
        2016.2327173169062
      ],
      "mutation_length": 0.0006515389664250358,
      "date": "2016-03-26",
      "clock_length": 0.0005188862530050666,
      "branch_length": 0.0005188862530050666
    },
    "V8375": {
      "raw_date": "2015-11-01",
      "numdate": 2015.8350444900752,
      "num_date_confidence": [
        2015.8350444900752,
        2015.8350444900752
      ],
      "mutation_length": 0.0008376315893888523,
      "date": "2015-10-31",
      "clock_

In addition to assigning times to internal nodes, the `refine` command filters tips that are likely outliers and assigns confidence intervals to inferred dates. Branch lengths in the resulting Newick tree measure adjusted nucleotide divergence. All other data inferred by TreeTime is stored by strain or internal node name in the corresponding JSON file.

# Annotate the Phylogeny
## Reconstruct Ancestral Traits
TreeTime can also infer ancestral traits from an existing phylogenetic tree and metadata annotating each tip of the tree. The following command infers the **region** and **country** of all internal nodes from the time tree and original strain metadata. As with the `refine` command, the resulting JSON output is indexed by strain or internal node name.

In [6]:
!augur traits \
  --tree zika-tutorial/results/tree.nwk \
  --metadata zika-tutorial/data/metadata.tsv \
  --output zika-tutorial/results/traits.json \
  --columns region country \
  --confidence


Inferred ancestral states of discrete character using TreeTime:
	Sagulenko et al. TreeTime: Maximum-likelihood phylodynamic analysis
	Virus Evolution, vol 4, https://academic.oup.com/ve/article/4/1/vex042/4794731

results written to zika-tutorial/results/traits.json


### Documentation for each command
#### `!augur traits`

`--tree zika-tutorial/results/tree.nwk`

`--metadata zika-tutorial/data/metadata.tsv`

`--output zika-tutorial/results/traits.json`

`--columns region country`: Metadata fields to perform discrete reconstruction on (default: None)

`--confidence`: Record the distribution of subleading migration states (default: False)

**Subleading migration states?**

### Output file traits.json

For each node:

```
"ZKC2/2016": {
      "region_entropy": -1.000088900581841e-12,
      "region_confidence": {
        "oceania": 1
      },
      "region": "oceania",
      "country_entropy": -1.000088900581841e-12,
      "country_confidence": {
        "american_samoa": 1
      },
      "country": "american_samoa"
    },
```


In [22]:
!cat zika-tutorial/results/traits.json | jq ''

{
  "nodes": {
    "ZKC2/2016": {
      "region_entropy": -1.000088900581841e-12,
      "region_confidence": {
        "oceania": 1
      },
      "region": "oceania",
      "country_entropy": -1.000088900581841e-12,
      "country_confidence": {
        "american_samoa": 1
      },
      "country": "american_samoa"
    },
    "VEN/UF_1/2016": {
      "region_entropy": -1.000088900581841e-12,
      "region_confidence": {
        "south_america": 1
      },
      "region": "south_america",
      "country_entropy": -1.000088900581841e-12,
      "country_confidence": {
        "venezuela": 1
      },
      "country": "venezuela"
    },
    "V8375": {
      "region_entropy": -1.000088900581841e-12,
      "region_confidence": {
        "north_america": 1
      },
      "region": "north_america",
      "country_entropy": -1.000088900581841e-12,
      "country_confidence": {
        "guatemala": 1
      },
      "country": "guatemala"
    },
    "USA/2016/FLWB042": {
      "region_entropy": -

## Infer Ancestral Sequences
Next, infer the ancestral sequence of each internal node and identify any nucleotide mutations on the branches leading to any node in the tree.

**What does a tree look like without ancestral sequences inferred? Aren't ancestral nodes inherently part of a phylogenetic tree?**

In [7]:
!augur ancestral \
  --tree zika-tutorial/results/tree.nwk \
  --alignment zika-tutorial/results/aligned.fasta \
  --output zika-tutorial/results/nt_muts.json \
  --inference joint

**WARNING: Number of sequences in tree differs from number of sequences in alignment!**

Inferred ancestral sequence states using TreeTime:
	Sagulenko et al. TreeTime: Maximum-likelihood phylodynamic analysis
	Virus Evolution, vol 4, https://academic.oup.com/ve/article/4/1/vex042/4794731

ancestral sequences written to zika-tutorial/results/nt_muts.json


### Documentation for each command
#### `!augur ancestral`

`--tree zika-tutorial/results/tree.nwk`

`--alignment zika-tutorial/results/aligned.fasta`

`--output zika-tutorial/results/nt_muts.json`

`--inference joint`: {joint,marginal} calculate joint or marginal maximum likelihood ancestral sequence states (default: joint)

**Again, joint vs marginal?**

### Output file nt_muts.json

For each node:
```
"nodes": {
    "VEN/UF_1/2016": {
      "sequence": [SEQUENCE],
      "muts": [
        "A3C",
        "G318T",
        "G438T",
        "C1233T",
        "C1416T",
        "A3373C",
        "C8016T",
        "G10147A"
      ]
    },
```

In [29]:
!cat zika-tutorial/results/nt_muts.json | jq ''

{
  "nodes": {
    "ZKC2/2016": {
      "sequence": "GAATCAGACTGCGACAGTTCGAGTTTGAAGCGAAAGCTAGCAACAGTATCAACAGGTTTTATTTGGATTTGGAAACGAGAGTTTCTGGTCATGAAAAACCCAAAAAAGAAATCCGGAGGATTCCGGATTGTCAATATGCTAAAACGCGGAGTAGCCCGTGTGAGCCCCTTTGGGGGCTTGAAGAGGCTGCCAGCCGGACTTCTGCTGGGTCATGGGCCCATCAGGATGGTCTTGGCGATTCTAGCCTTCTTGAGATTCACGGCAATCAAGCCATCACTGGGTCTCATCAATAGATGGGGTTCAGTGGGGAAAAAAGAGGCTATGGAAATAATAAAGAAGTTCAAGAAAGATCTGGCTGCCATGCTGAGAATAATCAATGCTAGGAAGGAGAAGAAGAGACGAGGCGCAGATACTAATGTCGGAATTGTTGGCCTCCTGCTGACCACAGCTATGGCAGCGGAGGTCACTAGACGTGGGAGTGCATACTATATGTACTTGGACAGAAACGATGCTGGGGAGGCCATATCTTTTCCAACCACATTGGGGATGAATAAGTGTTATATACAGATCATGGATCTTGGACACATGTGTGATGCCACCATGAGCTATGAATGCCCTATGCTGGATGAGGGGGTGGAACCAGATGACGTCGATTGTTGGTGCAACACGACGTCAACTTGGGTTGTGTACGGAACCTGCCATCACAAAAAAGGTGAAGCACGGAGATCTAGAAGAGCTGTGACGCTCCCCTCCCATTCCACTAGGAAGCTGCAAACGCGGTCGCAAACTTGGTTGGAATCAAGAGAATACACAAAGCACTTGATTAGAGTCGAAAATTGGATATTCAGGAACCCTGGCTTCGCGTTAGCAGCAGCTGCCATCGCTTGGCTTTTGGGAAGCTCAACGAGCCAAAAAGTCATATACTTGGTCATGATACTGCTGATTGC

      "sequence": "GAATCAGACTGCGACAGTTCGAGTTTGAAGCGAAAGCTAGCAACAGTATCAACAGGTTTTATTTGGATTTGGAAACGAGAGTTTCTGGTCATGAAAAACCCAAAAAAGAAATCCGGAGGATTCCGGATTGTCAATATGCTAAAACGCGGAGTAGCCCGTGTGAGCCCCTTTGGGGGCTTGAAGAGGCTGCCAGCCGGACTTCTGCTGGGTCATGGGCCCATCAGGATGGTCTTGGCGATTCTAGCCTTTTTGAGATTCACGGCAATCAAGCCATCACTGGGTCTCATCAATAGATGGGGTTCAGTGGGGAAAAAAGAGGCTATGGAAATAATAAAGAAGTTCAAGAAAGATCTGGCTGCCATGCTGAGAATAATCAATGCTAGGAAGGAGAAGAAGAGACGAGGCGCAGATACTAGTGTCGGAATTGTTGGCCTCCTGCTGACCACAGCTATGGCAGCGGAGGTCACTAGACGTGGGAGTGCATACTACATGTACTTGGACAGAAACGATGCTGGGGAGGCCATATCTTTCCCAACCACATTGGGGATGAATAAGTGTTATATACAGATCATGGATCTTGGACACATGTGTGATGCCACCATGAGCTATGAATGCCCTATGCTGGATGAGGGGGTGGAACCAGATGACGTCGATTGTTGGTGCAACACGACGTCAACTTGGGTTGTGTACGGAACCTGCCATCACAAAAAAGGTGAAGCACGGAGATCTAGAAGAGCTGTGACGCTCCCCTCCCATTCCACTAGGAAGCTGCAAACGCGGTCGCAAACCTGGTTGGAATCAAGAGAATACACAAAGCACTTGATTAGAGTCGAAAATTGGATATTCAGGAACCCTGGCTTCGCGTTAGCAGCAGCTGCCATCGCTTGGCTTTTGGGAAGCTCAACGAGCCAAAAAGTTATATACTTGGTCATGATACTGCTGATTGCCCCGGCATACAGCATCAGGTGCATAGGAGTCAGC

      "sequence": "GAATCAGACTGCGACAGTTCGAGTTTGAAGCGAAAGCTAGCAACAGTATCAACAGGTTTTATTTGGATTTGGAAACGAGAGTTTCTGGTCATGAAAAACCCAAAAAAGAAATCCGGAGGATTCCGGATTGTCAATATGCTAAAACGCGGAGTAGCCCGTGTGAGCCCCTTTGGGGGCTTGAAGAGGCTGCCAGCCGGACTTCTGCTGGGTCATGGGCCCATCAGGATGGTCTTGGCGATTCTAGCCTTTTTGAGATTCACGGCAATCAAGCCATCACTGGGTCTCATCAATAGATGGGGTTCAGTGGGGAAAAAAGAGGCTATGGAAATAATAAAGAAGTTCAAGAAAGATCTGGCTGCCATGCTGAGAATAATCAATGCTAGGAAGGAGAAGAAGAGACGAGGCGCAGATACTAGTGTCGGAATTGTTGGCCTCCTGCTGACCACAGCTATGGCAGCGGAGGTCACTAGACGTGGGAGTGCATACTATATGTACTTGGACAGAAACGATGCTGGGGAGGCCATATCTTTTCCAACCACATTGGGGATGAATAAGTGTTATATACAGATCATGGATCTTGGACACATGTGTGATGCCACCATGAGCTATGAATGCCCTATGCTGGATGAGGGGGTGGAACCAGATGACGTCGATTGTTGGTGCAACACGACGTCAACTTGGGTTGTGTACGGAACCTGCCATCACAAAAAAGGTGAAGCACGGAGATCTAGAAGAGCTGTGACGCTCCCCTCCCATTCCACTAGGAAGCTGCAAACGCGGTCGCAAACCTGGTTGGAATCAAGAGAATACACAAAGCACTTGATTAGAGTCGAAAATTGGATATTCAGGAACCCTGGCTTCGCGTTAGCAGCAGCTGCCATCGCTTGGCTTTTGGGAAGCTCAACGAGCCAAAAAGTCATATACTTGGTCATGATACTGCTGATTGCCCCGGCATACAGCATCAGGTGCATAGGAGTCAGC

## Identify Amino-Acid Mutations
Identify amino acid mutations from the nucleotide mutations and a reference sequence with gene coordinate annotations. The resulting JSON file contains amino acid mutations indexed by strain or internal node name and by gene name. To export a FASTA file with the complete amino acid translations for each gene from each node’s sequence, specify the `--alignment-output` parameter in the form of `results/aligned_aa_%GENE.fasta`.

In [8]:
!augur translate \
  --tree zika-tutorial/results/tree.nwk \
  --ancestral-sequences zika-tutorial/results/nt_muts.json \
  --reference-sequence zika-tutorial/config/zika_outgroup.gb \
  --output zika-tutorial/results/aa_muts.json

Read in 13 features from reference sequence file
amino acid mutations written to zika-tutorial/results/aa_muts.json


### Documentation for each command
#### `!augur translate`

`--tree zika-tutorial/results/tree.nwk`

`--ancestral-sequences zika-tutorial/results/nt_muts.json`: JSON (fasta input) or VCF (VCF input) containing ancestral and tip sequences (default: None)

`--reference-sequence zika-tutorial/config/zika_outgroup.gb`: GenBank or GFF file containing the annotation (default: None)

`--output zika-tutorial/results/aa_muts.json`

### Output file aa_muts.json

For each node:
```
"VEN/UF_1/2016": {
      "aa_muts": {
        "PRO": [],
        "NS5": [
          "A833T"
        ],
        "NS4B": [],
        "NS4A": [],
        "2K": [],
        "CA": [
          "E76D"
        ],
        "ENV": [],
        "MP": [],
        "NS1": [
          "T301P"
        ],
        "NS2A": [],
        "NS2B": [],
        "NS3": []
      }
    },
```

In [30]:
!cat zika-tutorial/results/aa_muts.json | jq ''

{
  "nodes": {
    "ZKC2/2016": {
      "aa_muts": {
        "PRO": [],
        "NS5": [],
        "NS4B": [],
        "NS4A": [],
        "2K": [],
        "CA": [],
        "ENV": [],
        "MP": [],
        "NS1": [],
        "NS2A": [],
        "NS2B": [],
        "NS3": []
      }
    },
    "VEN/UF_1/2016": {
      "aa_muts": {
        "PRO": [],
        "NS5": [
          "A833T"
        ],
        "NS4B": [],
        "NS4A": [],
        "2K": [],
        "CA": [
          "E76D"
        ],
        "ENV": [],
        "MP": [],
        "NS1": [
          "T301P"
        ],
        "NS2A": [],
        "NS2B": [],
        "NS3": []
      }
    },
    "V8375": {
      "aa_muts": {
        "PRO": [],
        "NS5": [
          "K174R"
        ],
        "NS4B": [],
        "NS4A": [],
        "2K": [],
        "CA": [],
        "ENV": [
          "V56I"
        ],
        "MP": [],
        "NS1": [],
        "NS2A": [],
        "NS2B": [],
        "NS3": []
      }
    },
    "USA/

# Below are the instructions to visualize the data with auspice

## Export the Results
Finally, collect all node annotations and metadata and export it all in auspice’s JSON format. This refers to three config files to define colors via `config/colors.tsv`, lat/long coordinates via `config/lat_longs.tsv` and page title, maintainer, filters present, etc… via `config/auspice_config.json`. The resulting tree and metadata JSON files are the inputs to the auspice visualization tool.

Copy the resulting .json files into your auspice/data folder

In [ ]:
!augur export \
  --tree zika-tutorial/results/tree.nwk \
  --metadata zika-tutorial/data/metadata.tsv \
  --node-data zika-tutorial/results/branch_lengths.json \
              zika-tutorial/results/traits.json \
              zika-tutorial/results/nt_muts.json \
              zika-tutorial/results/aa_muts.json \
  --colors zika-tutorial/config/colors.tsv \
  --lat-longs zika-tutorial/config/lat_longs.tsv \
  --auspice-config zika-tutorial/config/auspice_config.json \
  --output-tree auspice/zika_tree.json \
  --output-meta auspice/zika_meta.json

## Visualize the Results

I'm not sure how to do this from Jupyter notebook... These commands work if enterred in the command line.

In [13]:
!cd ~/auspice/data/ 
!npm run dev